# Extract Speeches from EU API

## Connect to Azure ML

In [7]:
from azureml.core import Workspace
ws = Workspace.from_config() # can be downloaded from Azure Machine Learning Workspace
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
#ws = Workspace.get(name="myworkspace",
#             subscription_id='<azure-subscription-id>',
#             resource_group='myresourcegroup')
#myenv = Environment.from_conda_specification(name = "myenv",                                           file_path = "requirements.txt")

Master1
Master
eastus2
491c115b-973e-4895-9b38-f849bc0cfd4d


In [25]:
from azureml.core.compute import ComputeTarget, AmlCompute
aml_compute_target = "master1-cluster"
aml_compute = AmlCompute(ws, aml_compute_target)
print("found existing compute target.")

found existing compute target.


In [8]:
from azureml.core import Experiment
experiment_name = 'train-on-local'
exp = Experiment(workspace=ws, name=experiment_name)

In [38]:
from azureml.core import Datastore
# Get file storage associated with the workspace
#def_file_store = Datastore(ws, "master1test")
#def_blob_store = Datastore(ws, "workspaceblobstore")
def_data_store = ws.get_default_datastore()

In [19]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

In [39]:
from azureml.core import Environment
myenv = Environment.from_pip_requirements(name = "myenv", file_path = "requirements.txt")
#from azureml.core import ScriptRunConfig
#src = ScriptRunConfig(source_directory='./scripts/01_download_year_info', script='01_download_year_info.py', environment=myenv)
#run = exp.submit(config=src)
#run.wait_for_completion(show_output=True)

#inputs=[input_ds.as_named_input('weather_ds')],
#compute_target=compute_target,
#outputs=[output_ds],

from azureml.pipeline.core import Pipeline, PipelineData
year_api_response = PipelineData("processed_data1",datastore=def_data_store)
#parliament_session_txts = PipelineData("processed_data2",datastore=def_file_store)

download_year_info = PythonScriptStep(
    name='Download_Year_Info',
    script_name='download.py', 
    arguments=["--output", year_api_response],
    compute_target=aml_compute,
    outputs=[year_api_response],
    source_directory='./scripts/01_download_year_info')

ingest_transcripts = PythonScriptStep(
    name='Download_Year_Info',
    script_name='ingest.py', 
    arguments=["--input_data", year_api_response, 
        "--output_data", parliament_session_txts,
        "--datastore_name","master1test"],
    inputs=[year_api_response],
    compute_target=aml_compute,
    outputs=[parliament_session_txts],
    source_directory='./scripts/02_ingest_transcripts')

# Use a RunConfiguration to specify some additional requirements for this step.
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# create a new runconfig object
run_config = RunConfiguration()

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['numpy'])


steps1 = [download_year_info]

pipeline = Pipeline(workspace=ws, steps=steps1)
pipeline_run1 = pipeline.submit("My-Experiment")

from azureml.widgets import RunDetails
RunDetails(pipeline_run1).show()





Created step Download_Year_Info [9935697e][3250c30d-9301-4c18-9dac-5b4dc920c58c], (This step will run and generate new outputs)
Submitted PipelineRun 5add64a1-cccb-4c86-ae32-f12b5638d5b7
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/My-Experiment/runs/5add64a1-cccb-4c86-ae32-f12b5638d5b7?wsid=/subscriptions/491c115b-973e-4895-9b38-f849bc0cfd4d/resourcegroups/Master/workspaces/Master1


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [5]:
from azureml.core import Environment
docker_env = Environment("docker-env")

docker_env.python.user_managed_dependencies = False
docker_env.docker.enabled = True


# use the default CPU-based Docker image from Azure ML
print(docker_env.docker.base_image)

# Specify conda dependencies with scikit-learn
#docker_env.python.conda_dependencies = cd

mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1


In [9]:
import subprocess

src.run_config.environment = docker_env

# Check if Docker is installed and Linux containers are enabled
if subprocess.run("docker -v", shell=True).returncode == 0:
    out = subprocess.check_output("docker system info", shell=True).decode('ascii')
    if not "OSType: linux" in out:
        print("Switch Docker engine to use Linux containers.")
    else:
        run = exp.submit(src)
        run.wait_for_completion(show_output = True)
else:
    print("Docker engine is not installed.")

RunId: train-on-local_1609368103_23b4af30
Web View: https://ml.azure.com/experiments/train-on-local/runs/train-on-local_1609368103_23b4af30?wsid=/subscriptions/491c115b-973e-4895-9b38-f849bc0cfd4d/resourcegroups/Master/workspaces/Master1

Streaming azureml-logs/60_control_log.txt

[2020-12-30T22:41:47.774934] Using urllib.request Python 3.0 or later
Streaming log file azureml-logs/60_control_log.txt
Starting the daemon thread to refresh tokens in background for process with pid = 80906
Running: ['/bin/bash', '/tmp/azureml_runs/train-on-local_1609368103_23b4af30/azureml-environment-setup/docker_env_checker.sh']

Materialized image not found on target: azureml/azureml_1a31af1e7a24bf75a2aeeda4db33c10b


[2020-12-30T22:41:48.457674] Logging experiment preparation status in history service.
Running: ['/bin/bash', '/tmp/azureml_runs/train-on-local_1609368103_23b4af30/azureml-environment-setup/docker_env_builder.sh']
Running: ['docker', 'build', '-f', 'azureml-environment-setup/Dockerfile', '